In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed .
# For example, here's several helpful packages to load .

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
!unzip '/content/drive/MyDrive/NEWS/fake or real news dection.zip' -d '/content/drive/MyDrive/NEWS/news_nlp'

Archive:  /content/drive/MyDrive/NEWS/fake or real news dection.zip
  inflating: /content/drive/MyDrive/NEWS/news_nlp/Fake.csv  
  inflating: /content/drive/MyDrive/NEWS/news_nlp/True.csv  


**In this notebook I want to classify fake news from true news.
I'm going to use Spacy en_core_web_lg model for word embeding and MultinomialNB and KNeighborsClassifier**

In [101]:
#loading true news dataset
df_true = pd.read_csv("/content/drive/MyDrive/NEWS/news_nlp/True.csv")
print(df_true.shape)
df_true.head()

(21417, 4)


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [102]:
#loading fake news dataset
df_fake = pd.read_csv("/content/drive/MyDrive/NEWS/news_nlp/Fake.csv")
print(df_fake.shape)
df_fake.head()

(23481, 4)


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [103]:
#insert label column
df_true['label'] = 1
df_fake['label'] = 0

In [104]:
import spacy.cli
spacy.cli.download('en_core_web_lg')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [105]:
#import spacy
nlp = spacy.load("en_core_web_lg")

Becuse the dataset is big I'm going to use only titles for classification

In [106]:
#transform title column from true news dataset to vectors
df_true['vector'] = df_true['title'].apply(lambda text:nlp(text).vector)
df_true.head()

,title,text,subject,date,label,vector
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1,"[-2.0529835, -0.569027, -3.1433282, 0.55970275..."
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1,"[-1.4419098, 2.333012, -2.0302525, 2.1737072, ..."
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1,"[0.557869, 2.2574153, 0.1913623, -0.041898414,..."
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1,"[-1.4035959, 0.07810003, -0.17810063, 2.512658..."
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1,"[0.010546171, 1.6784946, -0.5903046, 0.4184245..."


In [107]:
#transform title column from fake news dataset to vectors
df_fake['vector'] = df_fake['title'].apply(lambda text:nlp(text).vector)
df_fake.head()

,title,text,subject,date,label,vector
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0,"[0.82458675, -0.24166599, -0.066156715, 0.9113..."
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0,"[-0.106700815, -0.5254911, -0.10211553, 1.7716..."
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0,"[-0.005741093, -0.27893615, 2.1448169, -1.4584..."
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0,"[0.7094183, 0.55141747, 0.17757612, -0.1253749..."
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0,"[-0.74229336, -0.1589058, -0.97260255, 0.13083..."


In [108]:
#concatenate fake and true dataset
concatenated_df = pd.concat([df_true[['vector','label']], df_fake[['vector','label']]], ignore_index=True)
print(concatenated_df.shape)
concatenated_df.head()

(44898, 2)


,vector,label
0,"[-2.0529835, -0.569027, -3.1433282, 0.55970275...",1
1,"[-1.4419098, 2.333012, -2.0302525, 2.1737072, ...",1
2,"[0.557869, 2.2574153, 0.1913623, -0.041898414,...",1
3,"[-1.4035959, 0.07810003, -0.17810063, 2.512658...",1
4,"[0.010546171, 1.6784946, -0.5903046, 0.4184245...",1


In [109]:
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [110]:
#split the dataset to test and train part
X_train, X_test, y_train, y_test = train_test_split(concatenated_df['vector'], concatenated_df['label'], test_size=0.3
                                                    ,random_state=44)

X_train = np.stack(X_train)
X_test = np.stack(X_test)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((31428, 300), (31428,), (13470, 300), (13470,))

In [111]:
#making pipeline and train model number one using MinMaxScaler and MultinomialNB
clf1 = make_pipeline(
    MinMaxScaler(),
    MultinomialNB()
)

clf1.fit(X_train,y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('multinomialnb', MultinomialNB())])

In [112]:
print(classification_report(y_test, clf1.predict(X_test)))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      7136
           1       0.98      0.97      0.98      6334

    accuracy                           0.98     13470
   macro avg       0.98      0.98      0.98     13470
weighted avg       0.98      0.98      0.98     13470



In [113]:
#making pipeline and train model number one using MinMaxScaler and KNeighborsClassifier
clf2 = make_pipeline(
    MinMaxScaler(),
    KNeighborsClassifier(n_neighbors=5)
)

clf2.fit(X_train,y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('kneighborsclassifier', KNeighborsClassifier())])

In [114]:
print(classification_report(y_test, clf2.predict(X_test)))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7136
           1       1.00      0.99      0.99      6334

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [115]:
from sklearn.linear_model import LogisticRegression

In [116]:
#making pipeline and train model number one using MinMaxScaler and LogisticRegression
clf2 = make_pipeline(
    MinMaxScaler(),
    LogisticRegression())
clf2.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('logisticregression', LogisticRegression())])

In [117]:
print(classification_report(y_test, clf2.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7136
           1       1.00      1.00      1.00      6334

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



In [118]:
from sklearn.tree import DecisionTreeClassifier
#making pipeline and train model number one using MinMaxScaler and DecisionTreeClassifier
classfier = DecisionTreeClassifier()
minmax= MinMaxScaler()
clf2 = make_pipeline(
    minmax,classfier
    )
clf2.fit(X_train,y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [119]:
print(classification_report(y_test, clf2.predict(X_test)))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      7136
           1       0.99      0.98      0.98      6334

    accuracy                           0.98     13470
   macro avg       0.98      0.98      0.98     13470
weighted avg       0.98      0.98      0.98     13470



As we could see I get a very good result only using titles of news and with simple machine learning algorithm